# API 

## API v1/messages

```bash
curl https://api.anthropic.com/v1/messages \
     --header "x-api-key: $ANTHROPIC_API_KEY" \
     --header "anthropic-version: 2023-06-01" \
     --header "content-type: application/json" \
     --data \
'{
    "model": "claude-3-7-sonnet-20250219",
    "max_tokens": 1024,
    "messages": [
        {"role": "user", "content": "Hello, world"}
    ]
}'
```

### API demo

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
my_api_key = os.getenv("ANTHROPIC_API_KEY")


from anthropic import Anthropic

client = Anthropic(
    api_key=my_api_key,
    base_url = "http://localhost:11434", 
)

our_first_message = client.messages.create(
    model="llama3.2",
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "Hi there! Please write me a haiku about a pet chicken"}
    ]
)

print(our_first_message.content[0].text)

## messages format

The messages parameter expects a list of message dictionaries, where each dictionary represents a single message in the conversation. Each message dictionary should have the following keys:

role: A string indicating the role of the message sender. It can be either "user" (for messages sent by the user) or "assistant" (for messages sent by Claude).
content: A string or list of content dictionaries representing the actual content of the message. If a string is provided, it will be treated as a single text content block. If a list of content dictionaries is provided, each dictionary should have a "type" (e.g., "text" or "image") and the corresponding content. For now, we'll leave content as a single string.


![](./imdges/message_format.png)


The messages format allows us to structure our API calls to Claude in the form of a conversation, allowing for context preservation: The messages format allows for maintaining an entire conversation history, including both user and assistant messages. This ensures that Claude has access to the full context of the conversation when generating responses, leading to more coherent and relevant outputs.


## message response

The most important piece of information is the content property: this contains the actual content the model generated for us. This is a list of content blocks, each of which has a type that determines its shape.


![](./imdges/response_message_format.png)


In addition to content, the Message object contains some other pieces of information:

- id - a unique object identifier
- type - The object type, which will always be "message"
- role - The conversational role of the generated message. This will always be "assistant".
- model - The model that handled the request and generated the response
- stop_reason - The reason the model stopped generating. We'll learn more about this later.
- stop_sequence - We'll learn more about this shortly.
- usage - information on billing and rate-limit usage. Contains information on:
- input_tokens - The number of input tokens that were used.
- output_tokens - The number of output tokens that were used.


## Evaluations


A well-designed prompt evaluation consists of four primary components:

Example Input: This is the instruction or question given to the model. It's crucial to design prompts that accurately represent the kinds of inputs your application will encounter in real-world use.
Golden Answer: The correct or ideal response serves as a benchmark for the model's output. Creating high-quality golden answers often requires input from subject matter experts to ensure accuracy and relevance.
Model Output: This is the actual response generated by the LLM based on the input prompt. It's what you'll be evaluating against the golden answer.
Score: A quantitative or qualitative value representing the model's performance on that particular input. The scoring method can vary depending on the nature of your task and the grading approach you choose.


![](./imdges/evals.png)





## Key prompting tips


- Use the Prompt Generator
- Be clear and direct
- Use XML tags
- Use examples (multishot prompting)
- Let Claude think (chain of thought prompting)
- Give Claude a role (system prompts)
- Long context tips



## Tool

格式


In [ ]:
tool_definition = {
    "name": "get_stock_price",
    "description": "Retrieves the current stock price for a given company",
    "input_schema": {
        "type": "object",
        "properties": {
            "company": {
                "type": "string",
                "description": "The company name to fetch stock data for"
            }
        },
        "required": ["company"]
    }
}


def process_tool_call(tool_name, tool_input):
    """
    Process a tool call and return the result.
    """

messages = [{"role": "user", "content": "How many shares of General Motors can I buy with $500?"}]
response = client.messages.create(
    model="claude-3-opus-20240229",
    messages= messages,
    max_tokens=500,
    tools=[tool_definition]
)

# Update messages to include Claude's response
messages.append(
    {"role": "assistant", "content": response.content}
)

#If Claude stops because it wants to use a tool:
if response.stop_reason == "tool_use":
    tool_use = response.content[-1] #Naive approach assumes only 1 tool is called at a time
    tool_name = tool_use.name
    tool_input = tool_use.input
    print(f"======Claude wants to use the {tool_name} tool======")
    #Actually run the underlying tool functionality on our db
    tool_result = process_tool_call(tool_name, tool_input)
    #Add our tool_result message:
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": str(tool_result),
                }
            ],
        },
    )